In [1]:
%%html

<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

# Geographic Progression of The American Revolution


The American Revolution (1775-83) is also known as the American Revolutionary War and the U.S. War of Independence. 

Independence means not being under the control of others and being able to make one's own decisions. On July 4, 1776,  the Thirteen Colonies declared that they were no longer part of Britain. The British disagreed. A war between Britain and the Thirteen Colonies began as a result of this disagreement. This war was known as the American Revolution or the War of Independence. (From "Our Land and People", Patricia Shields-Ramsay, Douglas Ramsay)


> ![alt text](images/13_colonies.jpg)
>
> The location of the original Thirteen English Colonies on the east coast of what is now known as the United States of America. 

In order to understand the origins and progression of the American Revolutionary War, it can be interesting to look at the locations of various battles during each stage of the war. From this data, we can begin to understand where the battles were taking place, where they moved to, and try and understand the events surrounding them. To begin, we first need to download an open data set which contains the location and other information about the battles from Wikipedia into a table. The first few rows are shown below

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import nan as Nan
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
import time
init_notebook_mode(connected=True)

url = 'https://data.opendatasoft.com/explore/dataset/list-of-american-revolutionary-war-battles@public-us/download/?format=csv&timezone=America/Denver&use_labels_for_header=true'
revolution_data  = pd.read_csv(url,sep=';',error_bad_lines=False)
revolution_data.head()

,Battle,article,Date,State,Outcome,Start Date,Victory,Battle location
0,Battles of Lexington and Concord,https://en.wikipedia.org/wiki/Battles_of_Lexin...,"April 19, 1775",Massachusetts,Patriot victory: British forces raiding Concor...,1775-04-19,Patriot victory,"42.4496388889, -71.2308333333"
1,First Siege of Fort Henry,https://en.wikipedia.org/wiki/Siege_of_Fort_He...,NaN,Virginia,Patriot victory,NaN,Patriot victory,"40.0638888889, -80.725"
2,Battle of Freeman's Farm,https://en.wikipedia.org/wiki/Battles_of_Saratoga,"September 19, 1777",New York,British tactical victory: First of the two Bat...,1777-09-19,NaN,"42.9988888889, -73.6375"
3,Siege of Augusta,https://en.wikipedia.org/wiki/Siege_of_Augusta,NaN,Georgia,Patriot victory,NaN,Patriot victory,"33.4666666667, -81.9666666667"
4,Battle of Videau's Bridge,https://en.wikipedia.org/wiki/Battle_of_Videau...,"January 2, 1782",South Carolina,British victory,1782-01-02,British victory,NaN


From the table above we see that there is a lot of information to digest. In the first column, we have the name of the battle, and the second is a link to the Wikipedia page with information about the battle. We also have information like the date of the battle, the state it was located in, the consequences of that battle and who won. What might be new to some of you is the final column - the location of the battle which is represented by two numbers - latitude and longitude. Let's take a brief step back and explore what latitude and longitude represent on the globe.

## Brief Detour About Geography

**Latitudes and longitudes** are very basic terms in geography. They help us to explain the location or position of a something anywhere on the globe. Latitudes and longitudes form a grid system on the globe, and by knowing the shape of this grid, we an easily locate any place on the earth.

### Latitude

Latitudes are imaginary horizontal lines that encircle globe. Latitudes are sometimes called "parallels" as they are at an equal distance from each other and never meet, they are parallel lines!

Latitude lines measure how far north or south of the equator something is. The equator is defined as the starting point for measuring latitude - that's why it's marked as 0 degrees latitude. Latitude ranges from the equator at 0 degrees, to the north pole at 90 degrees, to the south pole at -90 degrees. The more "degrees latitude" describing somethings location means that it is further away from the equator. For example, something of 50 degrees latitude is further from the equator than something 20 degrees latitude. Depending on if something is above or below the equator they are given the directions north and south. Latitude locations are given as **degrees North** are positive (+) and latitude locations **degrees South** are negative (-). 


Each degree of latitude is approximately 111 kilometers apart (69 miles), although there is some variation as the earth is not perfectly round as it is shaped slightly like an egg. The globe below has the parallels drawn on it, feel free to spin the globe and zoom in or out on any location. On the globe below, parallels located south to the Equator are colored orange, this part of the Earth is called Southern Hemisphere. Conversely, parallels located north to the Equator are colored blue, and the part of the Earth  they cover is called Northern Hemisphere.


In [13]:
contours = []

equator_lon = [-180,-170,-160,-150,-140,-130,-120,-110,-100,-90,-80,-70,-60,-50,-40,-30,-20,-10,0,
               10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180]
euator_lat = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

#equator
contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= '0° equator',
        name = '0° equator',
        lon = equator_lon,
        lat = euator_lat,
        mode = 'lines',
        line = dict(
            width = 3,
            color = 'green'
        )
    ) )
#north pole
contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= '90° north pole',
        name = '90° north pole',
        lon = [0],
        lat = [90],
        mode = 'markers',
        line = dict(
            width = 2,
            color = 'rgb(50,136,189)'
        )
    ) )
#north hemispere
for i in range (1,9):
    contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= str(10*i)+'° north',
        name = str(10*i)+'° north',
        lon = equator_lon,
        lat = [x+10*i for x in euator_lat],
        mode = 'lines',
        line = dict(
            width = 2,
            color = 'rgb(50,136,189)'
        )
    ) )
    
#south hemispere
for i in range (1,9):
    contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= str(-10*i)+'° south',
        name = str(-10*i)+'° south',
        lon = equator_lon,
        lat = [x-10*i for x in euator_lat],
        mode = 'lines',
        line = dict(
            width = 2,
            color = 'rgb(244,109,67)'
        )
    ) )
    
contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= '-90° south pole',
        name = '-90° south pole',
        lon = [0],
        lat = [-90],
        mode = 'markers',
        line = dict(
            width = 2,
            color = 'rgb(244,109,67)'
        )
    ) )

layout = dict(
        title = 'Parallels over globe<br>(Click and drag to rotate)',        
        geo = dict(
            showland = True,
            showlakes = True,
            showcountries = True,
            showocean = True,
            countrywidth = 0.5,
            lakecolor = 'rgb(255, 255, 255)',
            oceancolor = 'rgb(255, 255, 255)',
            landcolor = 'rgb(212, 212, 212)',
            projection = dict( 
                type = 'orthographic',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )            
            ),

        )
    )
    
fig = dict( data=contours, layout=layout )
iplot( fig, validate=False )

### Questions

Look at the globe try rotating it and hover your mouse over different parallels. 
Every tenth parallel is displayed
If you click on the legend for any parallel on the left - that line will either appear or disappear on the map.

1. Can you find Equator? Which continents does it go through?
1. Find Canada. Is it located in Southern or Northern Hemisphere?
1. Between which parallels is Canada located?

### Longitude
In contrast to latitude, imaginary vertical lines on the globe are known as longitude. Longitude lines converge at poles and are widest at equator. Like latitude, longitude also sometimes get called by another name, this time they're called "meridian lines". Longitude lines are a way to show/measure how far a location is east or west of a universal vertical line called the Prime Meridian. This Prime Meridian line runs vertically, north and south, right over the British Royal Observatory in Greenwich England, from the North Pole to the South Pole. As the vertical starting point for longitude, the Prime Meridian is defined as 0 degrees longitude. Interestingly, for many years, each country had its own prime meridian. France used a meridian passing through Paris, while England used a meridian passing through Greenwich, England. Only in 1884, a group of scientists, navigators, and businessmen made Greenwich, England, the official worldwide Prime Meridian.

Longitude lines span 180 degrees both east and west of the Prime Meridian. Longitude lines that are **east** of the prime meridian are defined to be positive (+), and lines which are **west** of the Prime Meridian are defined to be negative (-). A special case of longitude is the longitude line which is 180 degrees both east and west of the Prime Meridian and it is known as the International Date Line. The International Date Line is directly opposite to the Prime Meridian. 

Lines of longitude at every ten degrees are shown on the globe below, try rotating it and hover your mouse over different meridians. If you click on the legend for any meridian on the left - if will disappear/reappear on the map. On the map below meridians located east to the Prime Meridian are colored blue, this part of the Earth is called Eastern Hemisphere. Conversely meridians located west to the Prime Meridian are colored orange, and this part of the Earth is called Western Hemisphere.

In [14]:
contours = []

mer_lat1 = [90,100,110,120,130,140,150,160,170,180,-180,-170,-160,-150,-140,-130,-120,-110,-100,-90]
mer_lon1 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

mer_lat2 = [-90,-80,-70,-60,-50,-40,-30,-20,-10,0,10,20,30,40,50,60,70,80,90]
mer_lon2 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

#zero meridian
contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= '0° Prime meridian',
        name = '0° Prime meridian',
        lon = mer_lon2,
        lat = mer_lat2,
        mode = 'lines',
        line = dict(
            width = 3,
            color = 'green'
        )
    ) )

contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= '180° International Date Line',
        name = '180° International Date Line',
        lon = mer_lon1,
        lat = mer_lat1,
        mode = 'lines',
        line = dict(
            width = 3,
            color = 'green'
        )
    ) )

#west hemispere
for i in range (1,18):
    contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= str(10*i-180)+'° west',
        name = str(10*i-180)+'° west',
        lat = mer_lat1,
        lon = [x+10*i for x in mer_lon1],
        mode = 'lines',
        line = dict(
            width = 2,
            color = 'rgb(244,109,67)'
        )
    ) )
    
#east hemispere
for i in range (1,18):
    contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= str(10*i)+'° east',
        name = str(10*i)+'° east',
        lat = mer_lat2,
        lon = [x+10*i for x in mer_lon2],
        mode = 'lines',
        line = dict(
            width = 2,
            color = 'rgb(50,136,189)'
        )
    ) )
    


layout = dict(
        title = 'Meridians over globe<br>(Click and drag to rotate)',
       # showlegend = False,         
        geo = dict(
            showland = True,
            showlakes = True,
            showcountries = True,
            showocean = True,
            countrywidth = 0.5,
            lakecolor = 'rgb(255, 255, 255)',
            oceancolor = 'rgb(255, 255, 255)',
            landcolor = 'rgb(212, 212, 212)',
            projection = dict( 
                type = 'orthographic',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )            
            ),
    
        )
    )
    
fig = dict( data=contours, layout=layout )
iplot( fig, validate=False )

### Questions



1. Can you locate Prime Meridian? 
    A. Is it passing through though Great Britain/London? 
    B. What other countries does the Prime Meridian pass through?
1. Locate Canada on the globe. Is Canada located in Eastern or Western Hemisphere?
1. Approximately what lines of latitude is Canada located between?

## Looking at Canada's map
If you completed the above questions, you should have noticed the following:
Canada is located between -50 and -140 degrees West (longitude) and between 42 and 83 degrees North (latitude).
Lets display some of the Canadian cities on the map.

From the map below, if you hover your mouse over different cities, their latitude and longitude locations are  displayed as a pair of numbers between brackets: `(longitude, latitude)`. For example, if you hover your mouse over our capital of Ottawa, you'll see the two numbers (-75.7, 45.42). This tells us that that Ottawa is located 75.7 degrees west (longitude)  and 45.42 degrees north (latitude).

In [15]:
import plotly.plotly as py
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df.head()
name=["Edmonton", "Victoria","Winnipeg","Fredericton","St. John's","Halifax","Toronto",
      #"Charlottetown","Quebec City", 
      "Regina","Whitehorse","Iqaluit","Yellowknife", "Ottawa"]
long=[-113.49,-123.37,-97.14,-66.64, -52.71, -63.57, -79.38,
      #-63.13,-71.21,
      -104.62,-135.06,-68.52,-114.38,-75.70]
latt=[53.54,48.43,49.90,45.97, 47.56, 44.65, 43.65,
      #46.238240,46.813878,
      50.44,60.72,63.75,62.45,45.42]
df['text'] = df['airport'] + '' + df['city'] + ', ' + df['state'] + '' + 'Arrivals: ' + df['cnt'].astype(str)

data = [ dict(
        type = 'scattergeo',
        locationmode = 'north america',
        lon = long,
        lat = latt,
        text = name,
        mode = 'markers+text',
        textposition= 'top',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            color = 'red',
        ))]

layout = dict(
        title = "Some Canada's provincial capitals <br>(Hover for coordinates)",
        colorbar = True,
        geo = dict(
            scope = 'north america',
            showland = True,
            landcolor = "rgb(212, 212, 212)",
            countrycolor = "rgb(255, 255, 255)",
            showlakes = True,
            lakecolor = "rgb(255, 255, 255)",
            showsubunits = False,
            showcountries = True,
            resolution = 50,
            projection = dict(
                type = 'kavrayskiy7',
            ),
            lonaxis = dict(
                showgrid = True,
                gridcolor ='rgb(244,109,67)',
                gridwidth = 2,
                range= [ -140, -50 ],
                dtick = 10
            ),
            lataxis = dict (
                showgrid = True,
                gridcolor ='rgb(50,136,189)',
                gridwidth = 2,
                range= [ 40.0, 80.0 ],
                dtick = 10
            )
        )
    )

fig = dict( data=data, layout=layout )
iplot( fig, validate=False )

### Questions 

1. Are there any negative values of latitude and longitude? Why or why not?
1. What's the northern most city on the map, what are its coordinates?
1. Locate both Winnipeg and Regina's, and note their coordinates coordinates. What do you notice about their latitude values? What does this tell you?
1. Locate both Edmonton and Yellowknife
    A. What is the difference between latitudes for Edmonton and Yellowknife?
    B. What does the distance tell you?
    
1. Calculate the distance between these cities given that 1 degree of latitude is covering approximately 111 km. 

(Answer: (62.45-53.54)*111=989.01km )


## Geographic View of The American Revolution

Now that we have a deeper understanding of the latitude and longitude values in our Revolutionary War data table  - lets go back to our dataset with American revolution battles.

Before we can put this data on a map, we first have to separate our latitude and longitude values from eachother! Let's split the **Battle location** column into two: one called "lon" for longitude and once called "lat" for latitude. When separated, these coordinates are much simpler to display on the map! 

 We are interested as well to look at battle by year  - so we need to take pull the year out of the **Start Date** column and put it in its own column in our data table. We will also remove table entries where we're missing values, displayed as `NaN` in the first table and filter out only North American battles. After making those modifications, the first few rows of our table can be seen below.

In [25]:
revolution_data = revolution_data.dropna(subset=['Battle location'])
revolution_data = revolution_data.dropna(subset=['Start Date'])
revolution_data['lat'] = revolution_data['Battle location'].str.split(', ', expand = True)[0]
revolution_data['lon'] = revolution_data['Battle location'].str.split(', ', expand = True)[1]
revolution_data['Year'] = revolution_data['Start Date'].str[:4]
revolution_data['lat']=pd.to_numeric(revolution_data['lat'], errors='coerce')
revolution_data['lon']=pd.to_numeric(revolution_data['lon'], errors='coerce')
revolution_data=revolution_data[revolution_data['lat']>12]
revolution_data=revolution_data[revolution_data['lat']<60]
revolution_data=revolution_data[revolution_data['lon']>-150]
revolution_data=revolution_data[revolution_data['lon']<-40]
revolution_data=revolution_data[revolution_data['Year'] != '2026']
revolution_data.loc[revolution_data.shape[0]]=[' ',' ',' ',' ',' ',' ','Patriot victory',' ','0','0','1783']

revolution_data.head()

,Battle,article,Date,State,Outcome,Start Date,Victory,Battle location,lat,lon,Year
0,Battles of Lexington and Concord,https://en.wikipedia.org/wiki/Battles_of_Lexin...,"April 19, 1775",Massachusetts,Patriot victory: British forces raiding Concor...,1775-04-19,Patriot victory,"42.4496388889, -71.2308333333",42.4496,-71.2308,1775
2,Battle of Freeman's Farm,https://en.wikipedia.org/wiki/Battles_of_Saratoga,"September 19, 1777",New York,British tactical victory: First of the two Bat...,1777-09-19,NaN,"42.9988888889, -73.6375",42.9989,-73.6375,1777
5,Mount Hope Bay raids,https://en.wikipedia.org/wiki/Mount_Hope_Bay_r...,"May 25-30, 1778",Rhode Island,British victory,1778-05-25,British victory,"41.694, -71.216",41.694,-71.216,1778
6,Capture of Savannah,https://en.wikipedia.org/wiki/Capture_of_Savannah,"December 29, 1778",Georgia,British victory,1778-12-29,British victory,"32.0508333333, -81.1038888889",32.0508,-81.1039,1778
12,Capture of Fort Bute,https://en.wikipedia.org/wiki/Capture_of_Fort_...,"September 7, 1779",West Florida,Patriot-Spanish victory,1779-09-07,NaN,"30.3236111111, -91.1369444444",30.3236,-91.1369,1779


We can now use this table to display the location of each battle easily on a map of North America. We will also colour the marker for the battle by which faction won the battle, and we will animate those battle by the year that they took place. The year can be controlled by adjusting the slider on the bottom of the map below, or by simply pressing the play button. Press the play button now to observe the progress of battles that took place during the American Civil war. You can also drag the slider over a specific year to see which battles took place that year individually. Finally, if you hover your mouse over the battle location, you can see the name of the battle, the location, and who won the battle. 

In [7]:
years_from_col = set(revolution_data['Year'])
years_ints = sorted(list(years_from_col))
years = [str(year) for year in years_ints]
revolution_data['Victory'] = revolution_data['Victory'].replace(np.nan, 'Unknown', regex=True)

victories = []

for victory in revolution_data['Victory']:
    if victory not in victories: 
        victories.append(victory)       

figure = {
    'data': [],
    'layout': {},
    'frames': []
}

figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1774',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}

custom_colors = {
    'Patriot victory': 'rgb(171, 0, 250)',
    'Unknown': 'rgb(150, 99, 0)',
    'British victory': 'rgb(99, 110, 250)',
}

figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}
figure['layout']['title'] = ' American Revolution (1775-83) battles'
figure['layout']['geo']  =  dict(
        scope = 'north america',
        showland = True,
        landcolor = "rgb(212, 212, 212)",
        subunitcolor = "rgb(255, 255, 255)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = True,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation = dict(
                lon = -100
            )
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
)
 

year = 1774
for victory in victories:
    dataset_by_year = revolution_data[revolution_data['Year'] == year]
    dataset_by_year_and_vict = dataset_by_year[dataset_by_year['Victory'] == victory]

    data_dict = {
        'type': 'scattergeo',
        'lon': list(dataset_by_year_and_vict['lon']),
        'lat': list(dataset_by_year_and_vict['lat']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_vict['Battle']),
        'marker': {
            'size': 8, 
            'color': custom_colors[victory]
        },
        'name': victory,
        'legendgroup' : 'group1'
    }
    figure['data'].append(data_dict)
    

for year in years:
    frame = {'data': [], 'name': str(year)}
    for victory in victories:
        dataset_by_year = revolution_data[revolution_data['Year'] == year]
        dataset_by_year_and_vict = dataset_by_year[dataset_by_year['Victory'] == victory]

        data_dict = {
            'type': 'scattergeo',
            'lon': list(dataset_by_year_and_vict['lon']),
            'lat': list(dataset_by_year_and_vict['lat']),
            'mode': 'markers',
            'text': list(dataset_by_year_and_vict['Battle']),
            'marker': {
                'size': 8, 
                'color': custom_colors[victory]
            },
            'name': victory,
            'legendgroup' : 'group'
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

    
figure['layout']['sliders'] = [sliders_dict]
iplot(figure, validate=False)

Using the interactive map above, you can observe the progression of the American Revolution in North America in terms of the battles fought each year. What may be more interesting to you as a Canadian student is to pay particular attention to the battles taking place in Quebec, Maritime provinces, and along the Saint Lawrence River. 

### Battle of Quebec - How it Shaped Canada?
We need a section demonstrating the Canadian connection to the war/how it helped form Canada. Unfortunately, I think this is going to take a day or two to research and do right. 




### Questions
1. Why were battles taking place in Quebec? 
1. Use the slider and hover over the battles on the map above, which battles were fought in what is now Canada?
1. Choose one of those battles, how might have that battle been important to shaping Canada as we know it today?



## The Progression of the War
While the map above makes it easy to see where and when various battles were fought during The American Revolution, sometimes it's easier to learn things from graphs and numbers. In the table below, we've used the same table as we used to create the map, however this time we've counted how many battles took place each year. Summarizing the table in this way allows us to quickly see which years in the war had the most battles.  

In [8]:
import colorlover as cl
results=revolution_data.groupby(['Year']).size().reset_index(name='Count').sort_values(by=['Count'])
results= results[results['Year']!=" "].reset_index(drop=True)
results["Colour"]=['rgb(255,255,255)'] + cl.scales['9']['seq']['Blues'] 
results=results.sort_values(by=['Year'])
#results = results.rename(columns={"Count":"Battles Fought"})
trace0 = go.Table(
  type = 'table',
  header = dict(
    values = [ "<b>YEAR</b>","<b>NUMBER OF BATTLES</b>",],
    line = dict(color = 'black'),
    fill = dict(color = 'white'),
    align = ['center'],
    font = dict(color = 'black', size = 12)
  ),
  cells = dict(
    values = [results.Year, results.Count],
    line = dict(color = 'rgb(0,0,0)'),
    fill = dict(color = 'rgb(255,255,255)'),
    align = 'center',
    height = 30,
    font = dict(color = 'rgb(0,0,0)', size = 20)
    ))

data = [trace0]

iplot(data)


From the table above, it's now clear that the year 1777 had the most battles in a year out of the entire war. The reason for this is that 1776-1777 was the beginning of the British counter offensive. This was a time frame where British forces made considerable progress against the Americans and ended up with several important victories, and captured thousands of American prisoners. 

### Questions
1. 1778 has about half as many battles as 1777, the year of the British counter offensive. What is the reason for this?
2. 1780 has the second most battles in a year, what events lead to this year having as many battles as it did?



## American Revolution Battle Locations

In the figure below we have again used our modified data table in order count which locations had the most battles during the war. In this case, we counted how many battles took place in each state/province/other location throughout the course of the war, and put it on a bar chart so we can easily see which locations had the most battles. The plot below is interactive - if you hover your mouse over a bar, it will display the number of battles that took place, as well as the percentage of battles that took place at that location.

In [26]:
results=revolution_data.groupby(['State']).size().reset_index(name='Count').sort_values(by=['Count'],ascending=False)
results= results[results['State']!=" "].reset_index(drop=True)
results['Percent'] = round(results['Count']*100/sum(results['Count']),2).astype(str) +"%"
trace0 = go.Bar(
    x=results['State'],
    y=results['Count'],
    text=results['Percent'],
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    opacity=0.6
)

data = [trace0]
layout = go.Layout(
    title='Number of battles by location',
   # font=dict(size=18),
    margin = dict(
    l= 30,
    r= 30,
    t= 50,
    b= 250
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

From the plot above, we see that the majority of battles took place in New York, South Carolina and New Jersey. This is another example where taking data that we can easily obtain online can be used to learn more about a subject that may not have been obvious without the use of computers. 

### Questions
1. What percentage of battles took place in New York, South Carolina and New Jersey?

## Summary

During our lesson about latitude and longitude and how they describe different locations on the globe, we also learned about how latitude and longitude information can help us learn about The American Revolution. Using latitude and longitude numbers describing the locations of battles, we were able to visualize the advancement of the Revolution. We were also able to discover how many battles took place each year, and which states or provinces had the most battles throughout the war. 


